# Midterm - Hao Hao

## Part 1:  SHORT, LET GET STARTED

Go to http://numbersapi.com and familiarize yourself with the API.

a) Write a program that accesses all trivia results for the numbers from 0 (zero) to 99 using batch requests only (One single query for all the numbers). Print the output result to the screen in the format of [3-digit number with leading zeros] - [TRIVIA] (One number per each line). E.g.,

000 - 0 is the coldest possible temperature old the Kelvin scale.   
001 - 1 is the number of dimensions of a line.   
002 - 2 is the price in cents per acre the USA bought Alaska from Russia.   
...   
Note: your output for each number may be different. The website randomizes its trivia responses.

In [1]:
import requests
from bs4 import BeautifulSoup
import re
import time
from selenium import webdriver
import json
import os
import pandas as pd
import numpy as np
import mysql.connector
from selenium.webdriver.common.keys import Keys

In [2]:
URL = "http://numbersapi.com/0..99/trivia"
headers = {'User-Agent': 'Mozilla/5.0'}
response = requests.get(URL, headers = headers)
result = response.json()

In [3]:
for i in range(0, len(result)):
    print(str(i).zfill(3) + ' - ' + result[str(i)])

000 - 0 is the coldest possible temperature old the Kelvin scale.
001 - 1 is the number of Gods in monotheism.
002 - 2 is the number of polynucleotide strands in a DNA double helix.
003 - 3 is the cost in cents to make a $1 bill in the United States.
004 - 4 is the number of human blood groups (A, B, O, AB).
005 - 5 is the holy number of Discordianism, as dictated by the Law of Fives.
006 - 6 is the number worn by LeBron James in the NBA.
007 - 7 is the number of seconds it takes "Superman: Escape from Krypton" roller coaster to go from 0 to 100 miles per hour.
008 - 8 is the number of furlongs in a mile.
009 - 9 is the number of circles of Hell in Dante's Divine Comedy.
010 - 10 is the number of Provinces in Canada.
011 - 11 is the number of pounds one gallon of pure maple syrup weighs.
012 - 12 is the number of keys in any standard digital telephone (1 through 9, 0, * and #).
013 - 13 is the number of steps in the gallows.
014 - 14 is the number of lines in a sonnet.
015 - 15 is the 

## Part 2: LONGER!

## Web-scraping: Which sellers advertise/sponsor on eBay

a) Take the code your group developed for assignment 2 and fix everything that may not have been optimal in your previous solution. Alternatively, but less cool, you may use our solution code as your starting point. (You will use this for the next part. You do not need to include this code in your submission.)

How would you modify your search query to only include buy-it-now (non-auction) items? What is the GET request's variable name corresponding to buy-it-now searches? How would you modify your search query to include 100 items per search result page? What is the GET request's variable name corresponding to items per page searches? Include your answers as a comment in your code marking it as Part 2-a. (This section does not involve coding)

In [4]:
## Modify my search query to only include buy-it-now items:
## https://www.ebay.com/sch/i.html?_nkw=samsung+tv&LH_BIN=1

## The GET request's variable name corresponding to buy-it-now searches:
## LH_BIN

## Modify my search query to include 100 items per search result page
## https://www.ebay.com/sch/i.html?_nkw=samsung+tv&_ipg=100

## The GET request's variable name corresponding to items per page searches:
## _ipg

## To make both changes:
## https://www.ebay.com/sch/i.html?_nkw=samsung+tv&LH_BIN=1&_ipg=100

## b) Save URLs of sponsored and non-sponsored items

Modify your previous code to search for buy-it-now listings of <keyword> and limit the number of items to 100 per page. Use this guide to select the right <keyword> assigned to you:

If your last name on Canvas starts with A-F use <keyword>: "canon eos 5d"

If your last name on Canvas starts with G-L use <keyword>: "playstation 4 slim"

If your last name on Canvas starts with M-R use <keyword>: "iphone 11 pro"

If your last name on Canvas starts with S-Z use <keyword>: "ipad 4"

Identify all the sponsored items.

For the first 10 pages of 100 items/page, save all the URLs of sponsored items' pages to the file "sponsored.txt"  and all the URLs of non-sponsored items' pages to the file "non-sponsored.txt" in the same directory as your code. (One URL per line in each file)

In [5]:
## Navigate to eBay's search result page for "playstation 4 slim"
URL = 'https://www.ebay.com/sch/i.html?_nkw=playstation+4+slim&LH_BIN=1&_ipg=100'

In [6]:
## Loop through all 10 pages and get the URLs for sponsored and non-sponsored items separately
sponsored_item_url = ''
non_sponsored_item_url = ''

for i in range(1, 11):
    responsei = requests.get(URL + "&_pgn=" + str(i), headers = headers)
    soupi = BeautifulSoup(responsei.content, 'html.parser')
    
    ## Find the part of code containing sponsorship information and the URL of each item
    link = soupi.find_all('a', class_ = "s-item__link")
    
    for j in range(0, len(link)):
        ## Find the sponsored items
        item = link[j].find_all('h3', class_ = "s-item__title s-item__title--has-tags")
        ## Get the URL of each item
        item_url = link[j].get('href')
        ## Simplify the URL
        item_url_simplify = re.sub(r'(.+)\?.*', r'\1', item_url)
        ## Separate sponsored and non-sponsored items
        if len(item) == 0:
            non_sponsored_item_url = non_sponsored_item_url + '\n' + item_url_simplify
        else:
            sponsored_item_url = sponsored_item_url + '\n' + item_url_simplify   
    
    ## Set a 2-second pause before going to the next page
    time.sleep(2)

In [7]:
## Save the URLs
with open('sponsored.txt', 'w') as file:
    file.write(sponsored_item_url.lstrip())
with open('non-sponsored.txt', 'w') as file:
    file.write(non_sponsored_item_url.lstrip())  

## c) Download pages of sponsored and non-sponsored items

Create two folders in the same directory as your code and name them "sponsored" and "non-sponsored". Write a program that opens the two files in (b) and downloads each of the pages (URLs) into the folders "sponsored" and "non-sponsored". Each file should be named as "item-id.htm" where you replace "item-id" with the ID of the item you are saving. E.g., "264616053293.htm" for the item with ID "264616053293". Note it is always good to put a 2-second pause between queries. Make sure to catch an error and continue if your query runs into problems connecting to eBay (e.g., if your internet is down for 5 seconds, you don't want your entire code to crash).

In [8]:
## Create two folders
path1 = 'sponsored'
os.mkdir(path1)
path2 = 'non-sponsored'
os.mkdir(path2)

In [9]:
## Open the two files
read_sponsored = open('sponsored.txt', 'r').read()
read_non_sponsored = open('non-sponsored.txt', 'r').read()

## Split the strings
sponsored_item_url_split = read_sponsored.split('\n')
non_sponsored_item_url_split = read_non_sponsored.split('\n')

In [10]:
## Download each sponsored item's URL into folder 'sponsored'
for item in range(0, len(sponsored_item_url_split)):
    URL = sponsored_item_url_split[item]
    response = requests.get(URL, headers = headers)
    iid = re.sub(r'.+/(\d{12})', r'\1', URL)
    
    with open(os.path.join(os.getcwd() + '/sponsored', str(iid) + '.htm'), 'w') as file:
        file.write(response.text)
    
    ## Set a 2-second pause before going to the next item
    time.sleep(2)

In [11]:
## Download each non-sponsored item's URL into folder 'non-sponsored'
for item in range(0, len(non_sponsored_item_url_split)):
    URL = non_sponsored_item_url_split[item]
    response = requests.get(URL, headers = headers)
    iid = re.sub(r'.+/(\d{12})', r'\1', URL)
    
    with open(os.path.join(os.getcwd() + '/non-sponsored', str(iid) + '.htm'), 'w') as file:
        file.write(response.text)
    
    ## Set a 2-second pause before going to the next item
    time.sleep(2)

## d) Select information

Write a separate piece of code that loops through the pages you downloaded in (c) and opens and parses them into a Python or Java xxxxsoup-object. Identify and select:

seller name, seller score, item price, # items sold, best offer available, title, returns allowed, shipping price, condition (e.g., used, new, like new, seller refurbished, ...).

In your code, highlight the selector command you choose to obtain each element using comments.

In [12]:
## Loop through sponsored pages
pages = os.listdir('./sponsored')
sponsored_df = pd.DataFrame(columns = ['sponsor', 'seller_name', 'seller_score', 'item_price', 'num_items_sold', 'best_offer_available', 'title', 'returns_allowed', 'shipping_price', 'condition'])
i = 0

for page in pages:
    readfile = open(os.path.join(os.getcwd() + '/sponsored', page), 'r').read()
    soup = BeautifulSoup(readfile, 'html.parser')
    
    ## identify sponsored or non-sponsored item
    sponsor = 'sponsored'
    
    ## seller name
    try:
        find_seller = soup.find('div', class_ = 'mbg vi-VR-margBtm3') ## selector command
        seller_name = find_seller.find('span', class_ = 'mbg-nw').get_text() ## selector command
    except:
        seller_name = None
    
    ## seller score
    try:
        seller_score = pd.to_numeric(find_seller.find('span', class_ = 'mbg-l').find('a').get_text()) ## selector command
    except:
        seller_score = None
    
    ## item price
    try:
        try:
            find_price = soup.find('span', id = 'prcIsum').get_text() ## selector command
            item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
        except:
            try:
                find_price = soup.find('span', id = 'convbidPrice').get_text() ## selector command
                item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
            except:
                try:
                    find_price = soup.find('span', id = 'mm-saleDscPrc').get_text() ## selector command
                    item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
                except:
                    try:
                        ## requires chromedriver and changing path below, otherwise use original price instead
                        driver = webdriver.Chrome('/Users/haohao/Desktop/Courses/422 - Data Design & Representation/chromedriver')
                        page_num = re.sub('.htm', '', page)
                        url = 'https://www.ebay.com/itm/' + str(page_num)
                        driver.get(url)
                        driver.find_element_by_id('e1').click()

                        soup2 = BeautifulSoup(driver.page_source, 'html.parser')

                        find_price = soup2.find('div', id = 'finalPrc').get_text()
                        item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
                        driver.close()
                    except:
                        find_price = soup.find('span', id = 'orgPrc').get_text() ## selector command
                        item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
    except:
        item_price = None
    
    ## number of items sold
    try:
        find_item = soup.find('div', class_ = 'nonActPanel') ## selector command
        sold = find_item.find('a', class_ = 'vi-txt-underline').get_text() ## selector command
        sold_num = re.sub(r'([,\d]+) sold', r'\1', sold)
        num_items_sold = pd.to_numeric(re.sub(r',', r'', sold_num))
    except:
        num_items_sold = None
    
    ## best offer available
    try:
        find_best_offer = soup.find('span', id = 'boBtn_btn_parent').get_text() ## selector command
        best_offer_available = 'Yes'
    except:
        best_offer_available = 'No'
    
    ## title
    try:
        title_text = soup.find('h1', id = 'itemTitle').get_text() ## selector command
        title = re.sub(r'Details about  \xa0', r'', title_text)
    except:
        title = None
    
    ## returns allowed
    try:
        find_returns = soup.find('span', id = 'vi-ret-accrd-txt').get_text() ## selector command
        if find_returns == 'Seller does not accept returns':
            returns_allowed = 'No'
        else:
            returns_allowed = 'Yes'
    except:
        returns_allowed = None
    
    ## shipping price
    try:
        try:
            find_shipping_price = soup.find('span', id = 'convetedPriceId').get_text() ## selector command
            shipping_price = pd.to_numeric(re.sub(r'[$\nUS]', r'', find_shipping_price))
        except:
            try:
                find_shipping_price = soup.find('span', id = 'fshippingCost').get_text() ## selector command
                try:
                    shipping_price = pd.to_numeric(re.sub(r'[$\n]', r'', find_shipping_price))
                except:
                    shipping_price = re.sub(r'[$\n]', r'', find_shipping_price)
            except:
                find_shipping_price = soup.find('span', id = 'shSummary').get_text() ## selector command
                try:
                    shipping_price = re.findall(r"FAST 'N FREE", find_shipping_price)[0]
                except:
                    try:
                        shipping_price = re.findall(r"Local pick-up offered.", find_shipping_price)[0]
                    except:
                        try:
                            shipping_price = re.sub(r'\n', r'', soup.find('span', id = 'fShippingSvc').get_text()) ## selector command
                        except:
                            shipping_price = soup.find('a', id = 'e3').get_text() ## selector command    
    except:
        shipping_price = None
    
    ## condition
    try:
        condition = soup.find('div', id = 'vi-itm-cond').get_text() ## selector command
    except:
        condition = None
    
    ## Save to data frame
    sponsored_df.loc[i] = [sponsor, seller_name, seller_score, item_price, num_items_sold, best_offer_available, title, returns_allowed, shipping_price, condition]
    i = i + 1

In [13]:
## Loop through non-sponsored pages
pages = os.listdir('./non-sponsored')
non_sponsored_df = pd.DataFrame(columns = ['sponsor', 'seller_name', 'seller_score', 'item_price', 'num_items_sold', 'best_offer_available', 'title', 'returns_allowed', 'shipping_price', 'condition'])
i = 0

for page in pages:
    readfile = open(os.path.join(os.getcwd() + '/non-sponsored', page), 'r').read()
    soup = BeautifulSoup(readfile, 'html.parser')
    
    ## identify sponsored or non-sponsored item
    sponsor = 'non-sponsored'
    
    ## seller name
    try:
        find_seller = soup.find('div', class_ = 'mbg vi-VR-margBtm3') ## selector command
        seller_name = find_seller.find('span', class_ = 'mbg-nw').get_text() ## selector command
    except:
        seller_name = None
    
    ## seller score
    try:
        seller_score = pd.to_numeric(find_seller.find('span', class_ = 'mbg-l').find('a').get_text()) ## selector command
    except:
        seller_score = None
    
    ## item price
    try:
        try:
            find_price = soup.find('span', id = 'prcIsum').get_text() ## selector command
            item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
        except:
            try:
                find_price = soup.find('span', id = 'convbidPrice').get_text() ## selector command
                item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
            except:
                try:
                    find_price = soup.find('span', id = 'mm-saleDscPrc').get_text() ## selector command
                    item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
                except:
                    try:
                        ## requires chromedriver and changing path below, otherwise use original price instead
                        driver = webdriver.Chrome('/Users/haohao/Desktop/Courses/422 - Data Design & Representation/chromedriver')
                        page_num = re.sub('.htm', '', page)
                        url = 'https://www.ebay.com/itm/' + str(page_num)
                        driver.get(url)
                        driver.find_element_by_id('e1').click()

                        soup2 = BeautifulSoup(driver.page_source, 'html.parser')

                        find_price = soup2.find('div', id = 'finalPrc').get_text()
                        item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
                        driver.close()
                    except:
                        find_price = soup.find('span', id = 'orgPrc').get_text() ## selector command
                        item_price = pd.to_numeric(re.sub(r'.*\$([.\d]+).*', r'\1', find_price))
    except:
        item_price = None
    
    ## number of items sold
    try:
        find_item = soup.find('div', class_ = 'nonActPanel') ## selector command
        sold = find_item.find('a', class_ = 'vi-txt-underline').get_text() ## selector command
        sold_num = re.sub(r'([,\d]+) sold', r'\1', sold)
        num_items_sold = pd.to_numeric(re.sub(r',', r'', sold_num))
    except:
        num_items_sold = None
    
    ## best offer available
    try:
        find_best_offer = soup.find('span', id = 'boBtn_btn_parent').get_text() ## selector command
        best_offer_available = 'Yes'
    except:
        best_offer_available = 'No'
    
    ## title
    try:
        title_text = soup.find('h1', id = 'itemTitle').get_text() ## selector command
        title = re.sub(r'Details about  \xa0', r'', title_text)
    except:
        title = None
    
    ## returns allowed
    try:
        find_returns = soup.find('span', id = 'vi-ret-accrd-txt').get_text() ## selector command
        if find_returns == 'Seller does not accept returns':
            returns_allowed = 'No'
        else:
            returns_allowed = 'Yes'
    except:
        returns_allowed = None
    
    ## shipping price
    try:
        try:
            find_shipping_price = soup.find('span', id = 'convetedPriceId').get_text() ## selector command
            shipping_price = pd.to_numeric(re.sub(r'[$\nUS]', r'', find_shipping_price))
        except:
            try:
                find_shipping_price = soup.find('span', id = 'fshippingCost').get_text() ## selector command
                try:
                    shipping_price = pd.to_numeric(re.sub(r'[$\n]', r'', find_shipping_price))
                except:
                    shipping_price = re.sub(r'[$\n]', r'', find_shipping_price)
            except:
                find_shipping_price = soup.find('span', id = 'shSummary').get_text() ## selector command
                try:
                    shipping_price = re.findall(r"FAST 'N FREE", find_shipping_price)[0]
                except:
                    try:
                        shipping_price = re.findall(r"Local pick-up offered.", find_shipping_price)[0]
                    except:
                        try:
                            shipping_price = re.sub(r'\n', r'', soup.find('span', id = 'fShippingSvc').get_text()) ## selector command
                        except:
                            shipping_price = soup.find('a', id = 'e3').get_text() ## selector command    
    except:
        shipping_price = None
    
    ## condition
    try:
        condition = soup.find('div', id = 'vi-itm-cond').get_text() ## selector command
    except:
        condition = None
  
    ## Save to data frame
    non_sponsored_df.loc[i] = [sponsor, seller_name, seller_score, item_price, num_items_sold, best_offer_available, title, returns_allowed, shipping_price, condition]
    i = i + 1

In [14]:
## Concatenate two tables
concat_df = sponsored_df.append(non_sponsored_df, ignore_index = True)
concat_df

,sponsor,seller_name,seller_score,item_price,num_items_sold,best_offer_available,title,returns_allowed,shipping_price,condition
0,sponsored,skateboardforever1986,2,320.00,None,Yes,WOW LOOK!! Sony PlayStation 4 Ps4 Slim 1tb Con...,No,FREE,Used
1,sponsored,productdeliveries,1693,192.79,None,Yes,(N05365) Sony Playstation 4 Slim 1TB Console,Yes,45.19,Used
2,sponsored,jasoyoun_39,56,339.00,None,Yes,Sony Playstation 4 Slim 500gb controller Bundle,No,Calculate,Manufacturer refurbished
3,sponsored,everyday_super_deals,635,579.99,10,Yes,NEW PlayStation 4 Pro 1TB Limited Edition Marv...,Yes,FREE,New
4,sponsored,sinwa1959,116,240.00,None,Yes,Sony PlayStation 4 Slim 1TB Console - Jet Blac...,No,Calculate,New
...,...,...,...,...,...,...,...,...,...,...
1054,non-sponsored,sansan66,1117,345.00,None,No,PlayStation 4 Slim 1TB Console Black + Fortnit...,No,FREE,New
1055,non-sponsored,Pawn America,166180,219.99,None,No,Sony PlayStation 4 Slim 1TB Console,Yes,FREE,Used
1056,non-sponsored,rosebeach10,30,340.00,None,No,Sony PlayStation 4 Slim Gaming Console,No,Calculate,New
1057,non-sponsored,2005amyx,258,304.99,None,Yes,Brand New & Sealed Sony PlayStation 4 Slim 1TB...,No,14.99,New


In [15]:
## Create a shipping_computed column for items of which shipping price need to be calaulated
shipping_computed = np.where(concat_df['shipping_price'] == 'Calculate', 'Yes', 'No')
concat_df.insert(9, 'shipping_computed', shipping_computed, True) 

In [16]:
## Clean the 'shipping_price' column
concat_df['shipping_price'] = np.where(concat_df['shipping_price'] == "FAST 'N FREE", 0, concat_df['shipping_price'])
concat_df['shipping_price'] = np.where(concat_df['shipping_price'] == 'FREE', 0, concat_df['shipping_price'])
concat_df['shipping_price'] = np.where(concat_df['shipping_price'] == 'Calculate', None, concat_df['shipping_price'])
concat_df['shipping_price'] = np.where(concat_df['shipping_price'] == 'Local pick-up offered.', 0, concat_df['shipping_price'])
concat_df['shipping_price'] = np.where(concat_df['shipping_price'] == 'Free Local Pickup', 0, concat_df['shipping_price'])

In [17]:
concat_df

,sponsor,seller_name,seller_score,item_price,num_items_sold,best_offer_available,title,returns_allowed,shipping_price,shipping_computed,condition
0,sponsored,skateboardforever1986,2,320.00,None,Yes,WOW LOOK!! Sony PlayStation 4 Ps4 Slim 1tb Con...,No,0,No,Used
1,sponsored,productdeliveries,1693,192.79,None,Yes,(N05365) Sony Playstation 4 Slim 1TB Console,Yes,45.19,No,Used
2,sponsored,jasoyoun_39,56,339.00,None,Yes,Sony Playstation 4 Slim 500gb controller Bundle,No,None,Yes,Manufacturer refurbished
3,sponsored,everyday_super_deals,635,579.99,10,Yes,NEW PlayStation 4 Pro 1TB Limited Edition Marv...,Yes,0,No,New
4,sponsored,sinwa1959,116,240.00,None,Yes,Sony PlayStation 4 Slim 1TB Console - Jet Blac...,No,None,Yes,New
...,...,...,...,...,...,...,...,...,...,...,...
1054,non-sponsored,sansan66,1117,345.00,None,No,PlayStation 4 Slim 1TB Console Black + Fortnit...,No,0,No,New
1055,non-sponsored,Pawn America,166180,219.99,None,No,Sony PlayStation 4 Slim 1TB Console,Yes,0,No,Used
1056,non-sponsored,rosebeach10,30,340.00,None,No,Sony PlayStation 4 Slim Gaming Console,No,None,Yes,New
1057,non-sponsored,2005amyx,258,304.99,None,Yes,Brand New & Sealed Sony PlayStation 4 Slim 1TB...,No,14.99,No,New


## e) Save information above into SQL database

Use your code script to connect to SQL (either MySQL, MariaDB, or SQLite. Do NOT use SQL GUI or command terminal). Create a database and name it "eBay". Save the information of items in (d)  into a single table named "eBay_items" (You are allowed to use only one table). This table should contain both sponsored and non-sponsored information and have a column that specifies which item is sponsored/non-sponsored.  If an item misses ANY of the information in (d), you should insert that missing value as NULL into the table. Convert any price (item price and shipping price) into a "dollar-cent" format (e.g., convert 12.34 into 1234 and 12 into 1200. Make sure the two least significant digits are cents. If an item does not include cents in the price, insert zeros.) and insert the price as INT into the table.

In [18]:
## Connect to local SQL instance
mydb = mysql.connector.connect(
  host="localhost",
  user="root",
  passwd=""
)
mycursor = mydb.cursor()
mycursor.execute("SET NAMES utf8mb4")

In [19]:
## Create a database named "eBay"
mycursor.execute("CREATE DATABASE IF NOT EXISTS eBay")

In [20]:
## Create a new SQL table named "eBay_items"
mycursor.execute("USE eBay")

mycursor.execute("CREATE TABLE IF NOT EXISTS eBay_items (sponsor VARCHAR(255), seller_name VARCHAR(255), seller_score INT, item_price INT, num_items_sold INT, best_offer_available VARCHAR(255), title VARCHAR(255), returns_allowed VARCHAR(255), shipping_price INT, shipping_computed VARCHAR(255), item_condition VARCHAR(255))")

In [21]:
for r in concat_df.index:
    
    ## Change item_price to cents
    try:
        item_price = int(concat_df['item_price'][r] * 100)
    except:
        item_price = None
    
    ## Change shipping_price to cents
    try:
        shipping_price = int(concat_df['shipping_price'][r] * 100)
    except:
        shipping_price = None
    
    ## Insert data into table 'eBay_items'
    sql = "INSERT INTO eBay_items (sponsor, seller_name, seller_score, item_price, num_items_sold, best_offer_available, title, returns_allowed, shipping_price, shipping_computed, item_condition) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    val = (concat_df['sponsor'][r], concat_df['seller_name'][r], concat_df['seller_score'][r], item_price, concat_df['num_items_sold'][r], concat_df['best_offer_available'][r], concat_df['title'][r], concat_df['returns_allowed'][r], shipping_price, concat_df['shipping_computed'][r], concat_df['condition'][r])
    mycursor.execute(sql, val)
    mydb.commit()

## f) Summary

Use your code script (and NOT SQL GUI or command terminal) to run summary stats on each item. Print to the screen the mean, min, max, and mean for each column, grouped by "sponsor/non-sponsor" and "condition" (group by at the same time, not separately). For binary categorical columns, use 0-1 conversion. For e.g., for the "returns allowed" convert YES to 1 and NO to 0 and then calculate the stats. If it is NOT a numerical/binary categorical column, print to the screen the count of each category level. You will need to ignore NULL values in your statistic calculations. 

In [22]:
## Use 0-1 conversion for the binary categorical column 'returns_allowed'
concat_df['returns_allowed'] = np.where(concat_df['returns_allowed'] == 'Yes', 1, concat_df['returns_allowed'])
concat_df['returns_allowed'] = np.where(concat_df['returns_allowed'] == 'No', 0, concat_df['returns_allowed'])

In [23]:
## Use 0-1 conversion for the binary categorical column 'best_offer_available'
concat_df['best_offer_available'] = np.where(concat_df['best_offer_available'] == 'Yes', 1, concat_df['best_offer_available'])
concat_df['best_offer_available'] = np.where(concat_df['best_offer_available'] == 'No', 0, concat_df['best_offer_available'])

In [24]:
## Use 0-1 conversion for the binary categorical column 'shipping_computed'
concat_df['shipping_computed'] = np.where(concat_df['shipping_computed'] == 'Yes', 1, concat_df['shipping_computed'])
concat_df['shipping_computed'] = np.where(concat_df['shipping_computed'] == 'No', 0, concat_df['shipping_computed'])

In [25]:
## Convert to numerical
concat_df['seller_score'] = pd.to_numeric(concat_df['seller_score'])
concat_df['num_items_sold'] = pd.to_numeric(concat_df['num_items_sold'])
concat_df['best_offer_available'] = pd.to_numeric(concat_df['best_offer_available'])
concat_df['returns_allowed'] = pd.to_numeric(concat_df['returns_allowed'])
concat_df['shipping_price'] = pd.to_numeric(concat_df['shipping_price'])
concat_df['shipping_computed'] = pd.to_numeric(concat_df['shipping_computed'])

In [26]:
## Build the summary table structure
summary = concat_df[['sponsor', 'condition', 'item_price']].groupby(['sponsor', 'condition'], as_index = False).mean()
summary = summary.drop('item_price', 1)

In [27]:
## Summary stats for each numerical column
numerical = ['seller_score', 'item_price', 'num_items_sold', 'best_offer_available', 'returns_allowed', 'shipping_price', 'shipping_computed']

for i in numerical:
    col_mean = concat_df[['sponsor', 'condition', i]].groupby(['sponsor', 'condition'], as_index = False).mean()
    col_mean.rename(columns = {i: 'mean_' + i}, inplace = True)

    col_min = concat_df[['sponsor', 'condition', i]].groupby(['sponsor', 'condition'], as_index = False).min()
    col_min.rename(columns = {i: 'min_' + i}, inplace = True)

    col_max = concat_df[['sponsor', 'condition', i]].groupby(['sponsor', 'condition'], as_index = False).max()
    col_max.rename(columns = {i: 'max_' + i}, inplace = True)
    
    summary = pd.merge(summary, col_mean, on = ['sponsor', 'condition'])
    summary = pd.merge(summary, col_min, on = ['sponsor', 'condition'])
    summary = pd.merge(summary, col_max, on = ['sponsor', 'condition'])

summary

,sponsor,condition,mean_seller_score,min_seller_score,max_seller_score,mean_item_price,min_item_price,max_item_price,mean_num_items_sold,min_num_items_sold,...,max_best_offer_available,mean_returns_allowed,min_returns_allowed,max_returns_allowed,mean_shipping_price,min_shipping_price,max_shipping_price,mean_shipping_computed,min_shipping_computed,max_shipping_computed
0,non-sponsored,Manufacturer refurbished,452838.400000,0,2264137,213.598000,190.00,250.00,NaN,NaN,...,1,0.400000,0,1,0.000000,0.0,0.00,0.800000,0,1
1,non-sponsored,New,12054.171014,-1,255927,311.843304,1.00,699.99,41.276316,1.0,...,1,0.524638,0,1,2.823773,0.0,48.54,0.208696,0,1
2,non-sponsored,Open box,23859.827586,0,638346,276.808276,190.00,529.95,6.000000,1.0,...,1,0.189655,0,1,3.903810,0.0,25.00,0.637931,0,1
3,non-sponsored,Seller refurbished,281428.676471,0,2264137,225.876176,159.99,349.00,133.727273,1.0,...,1,0.588235,0,1,2.707917,0.0,29.99,0.294118,0,1
4,non-sponsored,Used,53157.545090,0,224373,230.762285,144.99,520.00,8.571429,1.0,...,1,0.509018,0,1,3.502663,0.0,46.60,0.292585,0,1
5,sponsored,Manufacturer refurbished,197.333333,3,533,261.330000,199.99,339.00,3.000000,3.0,...,1,0.333333,0,1,0.000000,0.0,0.00,0.666667,0,1
6,sponsored,New,35261.717391,0,916666,339.189348,221.00,599.99,45.500000,1.0,...,1,0.652174,0,1,4.113429,0.0,49.99,0.239130,0,1
7,sponsored,Open box,15817.800000,5,64474,257.600000,195.00,335.00,5.000000,5.0,...,1,0.400000,0,1,5.000000,0.0,15.00,0.400000,0,1
8,sponsored,Seller refurbished,15090.000000,0,41447,229.995000,199.99,253.00,855.000000,4.0,...,1,0.750000,0,1,0.000000,0.0,0.00,0.250000,0,1
9,sponsored,Used,18719.116667,0,258522,268.111167,150.00,695.39,444.250000,8.0,...,1,0.500000,0,1,11.995714,0.0,52.72,0.300000,0,1


In [28]:
## Count of each level for not numerical/binary categorical columns
factors = ['seller_name', 'title']
for i in factors:
    col_count = concat_df[['sponsor', 'condition', i]].groupby(['sponsor', 'condition'], as_index = False).count()
    col_count.rename(columns={i: 'count_' + i}, inplace = True)
    
    summary = pd.merge(summary, col_count, on = ['sponsor', 'condition'])

summary

,sponsor,condition,mean_seller_score,min_seller_score,max_seller_score,mean_item_price,min_item_price,max_item_price,mean_num_items_sold,min_num_items_sold,...,min_returns_allowed,max_returns_allowed,mean_shipping_price,min_shipping_price,max_shipping_price,mean_shipping_computed,min_shipping_computed,max_shipping_computed,count_seller_name,count_title
0,non-sponsored,Manufacturer refurbished,452838.400000,0,2264137,213.598000,190.00,250.00,NaN,NaN,...,0,1,0.000000,0.0,0.00,0.800000,0,1,5,5
1,non-sponsored,New,12054.171014,-1,255927,311.843304,1.00,699.99,41.276316,1.0,...,0,1,2.823773,0.0,48.54,0.208696,0,1,345,345
2,non-sponsored,Open box,23859.827586,0,638346,276.808276,190.00,529.95,6.000000,1.0,...,0,1,3.903810,0.0,25.00,0.637931,0,1,58,58
3,non-sponsored,Seller refurbished,281428.676471,0,2264137,225.876176,159.99,349.00,133.727273,1.0,...,0,1,2.707917,0.0,29.99,0.294118,0,1,34,34
4,non-sponsored,Used,53157.545090,0,224373,230.762285,144.99,520.00,8.571429,1.0,...,0,1,3.502663,0.0,46.60,0.292585,0,1,499,499
5,sponsored,Manufacturer refurbished,197.333333,3,533,261.330000,199.99,339.00,3.000000,3.0,...,0,1,0.000000,0.0,0.00,0.666667,0,1,3,3
6,sponsored,New,35261.717391,0,916666,339.189348,221.00,599.99,45.500000,1.0,...,0,1,4.113429,0.0,49.99,0.239130,0,1,46,46
7,sponsored,Open box,15817.800000,5,64474,257.600000,195.00,335.00,5.000000,5.0,...,0,1,5.000000,0.0,15.00,0.400000,0,1,5,5
8,sponsored,Seller refurbished,15090.000000,0,41447,229.995000,199.99,253.00,855.000000,4.0,...,0,1,0.000000,0.0,0.00,0.250000,0,1,4,4
9,sponsored,Used,18719.116667,0,258522,268.111167,150.00,695.39,444.250000,8.0,...,0,1,11.995714,0.0,52.72,0.300000,0,1,60,60


## g) Difference between sponsored and non-sponsored items

Use the stats in (f) and tell in your own words, how sponsored and non-sponsored items appear to be different. Is there a variable/column that can be used to predict the sponsor/non-sponsor items. (You do NOT need to run any model or statistic calculations for this part. Make your judgment only by looking at the stats you printed in (f))

#### Answer:    
Sponsored items generally tend to have lower mean seller score, higher mean number of items sold, higher percentage of mean returns allowed, higher mean shipping price, and lower number of listings.   
Based on the differences above, seller score can be used to predict the sponsor/non-sponsor items, because the difference of that seems the most obvious among all variables.